*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     |████████████████████████████████| 489.8 MB 25 kB/s 
     |████████████████████████████████| 2.1 MB 38.7 MB/s 
     |████████████████████████████████| 1.3 MB 49.6 MB/s 
     |████████████████████████████████| 13.4 MB 241 kB/s 
     |████████████████████████████████| 5.8 MB 18.4 MB/s 
     |████████████████████████████████| 463 kB 77.1 MB/s 
2.8.0-dev20211004


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-10-05 03:14:32--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.004s  

2021-10-05 03:14:32 (95.6 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2021-10-05 03:14:32--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
train = pd.read_csv(train_file_path, sep='\t', header=None)
train.rename(columns={0:'label', 1:'message'}, inplace=True)
train.dropna()
train.label.replace({'ham':0, 'spam':1}, inplace=True)

test = pd.read_csv(test_file_path, sep='\t', header=None)
test.rename(columns={0:'label', 1:'message'}, inplace=True)
test.dropna()
test.label.replace({'ham':0, 'spam':1}, inplace=True)

In [ ]:
train.head()

,label,message
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
test.head()

,label,message
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?


In [ ]:
test.shape

(1392, 2)

In [ ]:
train.shape

(4179, 2)

In [ ]:
train_label = train.pop('label')
test_label = test.pop('label')

In [ ]:
train

,message
0,ahhhh...just woken up!had a bad dream about u ...
1,you can never do nothing
2,"now u sound like manky scouse boy steve,like! ..."
3,mum say we wan to go then go... then she can s...
4,never y lei... i v lazy... got wat? dat day ü ...
...,...
4174,just woke up. yeesh its late. but i didn't fal...
4175,what do u reckon as need 2 arrange transport i...
4176,free entry into our £250 weekly competition ju...
4177,-pls stop bootydelious (32/f) is inviting you ...


In [ ]:
train_label

0       0
1       0
2       0
3       0
4       0
       ..
4174    0
4175    0
4176    1
4177    1
4178    0
Name: label, Length: 4179, dtype: int64

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train.message, train_label))
for i in train_data.take(2):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'you can never do nothing'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [ ]:
test_data = tf.data.Dataset.from_tensor_slices((test.message, test_label))
for i in test_data.take(2):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'i am in hospital da. . i will return home in evening'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"not much, just some textin'. how bout you?">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_dataset = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'i\xc2\x92m cool ta luv but v.tired 2 cause i have been doin loads of planning all wk, we have got our social services inspection at the nursery! take care & spk sn x.'
 b'congrats! 2 mobile 3g videophones r yours. call 09063458130 now! videochat wid your mates, play java games, dload polyph music, noline rentl.'
 b'keep my payasam there if rinu brings']

labels:  [0 1 0]


In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U15')

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
131/131 [==============================] - 18s 139ms/step - loss: 0.0201 - accuracy: 0.9957 - val_loss: 0.0595 - val_accuracy: 0.9906
Epoch 2/10
131/131 [==============================] - 19s 142ms/step - loss: 0.0149 - accuracy: 0.9971 - val_loss: 0.0681 - val_accuracy: 0.9885
Epoch 3/10
131/131 [==============================] - 18s 140ms/step - loss: 0.0111 - accuracy: 0.9978 - val_loss: 0.0658 - val_accuracy: 0.9896
Epoch 4/10
131/131 [==============================] - 19s 143ms/step - loss: 0.0093 - accuracy: 0.9981 - val_loss: 0.0672 - val_accuracy: 0.9896
Epoch 5/10
131/131 [==============================] - 19s 144ms/step - loss: 0.0071 - accuracy: 0.9986 - val_loss: 0.0708 - val_accuracy: 0.9896
Epoch 6/10
131/131 [==============================] - 19s 140ms/step - loss: 0.0067 - accuracy: 0.9986 - val_loss: 0.0729 - val_accuracy: 0.9896
Epoch 7/10
131/131 [==============================] - 18s 139ms/step - loss: 0.0053 - accuracy: 0.9986 - val_loss: 0.0764 - val_ac

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

44/44 [==============================] - 1s 32ms/step - loss: 0.0777 - accuracy: 0.9864
Test Loss: 0.07771727442741394
Test Accuracy: 0.9863505959510803


In [ ]:
# predict on a sample text

sample_text = ('call the 99855')
predictions = model.predict(np.array([sample_text]))
print(predictions)

[[-1.2172271]]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  predict = model.predict(np.array([pred_text]))
  prediction = [predict[0][0], 'ham' if predict[0][0]<0.5 else 'spam' ]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[-12.635724, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
